In [71]:
import pandas as pd
import numpy as np
import pingouin as pg
from matplotlib import pyplot as plt 
import seaborn as sns

In [72]:
behdata = pd.read_csv('BehaData/fmri_behavioural_new.csv')
behdata = behdata.dropna(subset=['obs_p1g2', 'obs_p2g1'], how='all')
print(behdata.head())

    subject  session  trial  seq        rt  gen_p1g2  gen_p2g1  prob_pxl  \
21        6        1     22    2  5.318199      0.75      0.40     690.0   
42        6        1     43    1  4.738264      0.75      0.40     700.0   
60        6        1     61    1  4.147888      0.75      0.40     755.0   
79        6        1     80    2  3.889652      0.35      0.85     735.0   
98        6        1     99    2  3.973970      0.35      0.85     690.0   

    prob_obs  obs_p1g2  obs_p2g1  obs_p1g1  obs_p2g2    obs_p1    obs_p2  \
21  0.800000       0.2       NaN       NaN       0.8  0.200000  0.800000   
42  0.733333       NaN  0.266667  0.733333       NaN  0.733333  0.266667   
60  0.366667       NaN  0.633333  0.366667       NaN  0.366667  0.633333   
79  0.500000       0.5       NaN       NaN       0.5  0.500000  0.500000   
98  0.800000       0.2       NaN       NaN       0.8  0.200000  0.800000   

       obs_p  runtime  
21  0.200000    40.25  
42  0.266667    78.90  
60  0.633333  

In [73]:
behdata["gen_p1g1"] = 1 - behdata["gen_p2g1"]
behdata["gen_p2g2"] = 1 - behdata["gen_p1g2"]
behdata["gen_p1"] = (behdata["gen_p1g1"] + behdata["gen_p1g2"])/(behdata["gen_p1g1"] + behdata["gen_p1g2"] + behdata["gen_p2g1"] + behdata["gen_p2g2"])
behdata["gen_p2"] = (behdata["gen_p2g1"] + behdata["gen_p2g2"])/(behdata["gen_p1g1"] + behdata["gen_p1g2"] + behdata["gen_p2g1"] + behdata["gen_p2g2"])

# 被试表现出来的转移系数和实际设定的转移系数之间的相关

## L|H

In [74]:
# 在 L|H 上的设定的转移系数和被试反应的转移系数的相关状况
DataLP = behdata[['gen_p1g2', 'obs_p1g2','subject']].dropna()
DataLP_sublevel = DataLP.groupby(['subject']).transform(lambda x: (x - x.mean()) / x.std())
DataLP_sublevel["subject"] = DataLP["subject"]
DataLP_sublevel = DataLP_sublevel.rename(
    columns={'gen_p1g2':'Generative p(L|H)',
             'obs_p1g2':'Observed p(L|H)'})
print(pg.corr(
    x=DataLP_sublevel['Generative p(L|H)'], 
    y=DataLP_sublevel['Observed p(L|H)'], 
    method='pearson'))

           n         r         CI95%     p-val     BF10     power
pearson  983  0.131399  [0.07, 0.19]  0.000036  201.487  0.985348


In [75]:
# 对每个被试分别做相关，看看多少被试的相关系数大于0
r_ls = []
for subid in behdata["subject"].unique():
    subdata = DataLP[DataLP["subject"] == subid]
    corr = pg.corr(
        x=subdata['gen_p1g2'], 
        y=subdata['obs_p1g2'], 
        method='pearson')
    r_ls.append(corr['r'].values[0])
print(len([r for r in r_ls if r > 0])/len(r_ls))

0.7428571428571429


## H|H

In [76]:
# 在 H|H 上的设定的转移系数和被试反应的转移系数的相关状况
DataHH = behdata[['gen_p2g2', 'obs_p2g2','subject']].dropna()
DataHH_sublevel = DataHH.groupby(['subject']).transform(lambda x: (x - x.mean()) / x.std())
DataHH_sublevel["subject"] = DataHH["subject"]
DataHH_sublevel = DataHH_sublevel.rename(
    columns={'gen_p2g2':'Generative p(H|H)',
             'obs_p2g2':'Observed p(H|H)'})
print(pg.corr(
    x=DataHH_sublevel['Generative p(H|H)'], 
    y=DataHH_sublevel['Observed p(H|H)'], 
    method='pearson'))

           n         r         CI95%     p-val     BF10     power
pearson  983  0.131399  [0.07, 0.19]  0.000036  201.487  0.985348


In [77]:
r_ls = []
for subid in behdata["subject"].unique():
    subdata = DataHH[DataHH["subject"] == subid]
    corr = pg.corr(
        x=subdata['gen_p2g2'], 
        y=subdata['obs_p2g2'], 
        method='pearson')
    r_ls.append(corr['r'].values[0])
print(len([r for r in r_ls if r > 0])/len(r_ls))

0.7428571428571429


## H|L

In [78]:
# 在 H|L 上的设定的转移系数和被试反应的转移系数的相关状况
DataHL = behdata[['gen_p2g1', 'obs_p2g1','subject']].dropna()
DataHL_sublevel = DataHL.groupby(['subject']).transform(lambda x: (x - x.mean()) / x.std())
DataHL_sublevel["subject"] = DataHL["subject"]
DataHL_sublevel = DataHL_sublevel.rename(
    columns={'gen_p2g1':'Generative p(H|L)',
             'obs_p2g1':'Observed p(H|L)'})
print(pg.corr(
    x=DataHL_sublevel['Generative p(H|L)'], 
    y=DataHL_sublevel['Observed p(H|L)'], 
    method='pearson'))

            n         r         CI95%     p-val     BF10     power
pearson  1121  0.118704  [0.06, 0.18]  0.000068  103.567  0.978798


In [79]:
r_ls = []
for subid in behdata["subject"].unique():
    subdata = DataHL[DataHL["subject"] == subid]
    corr = pg.corr(
        x=subdata['gen_p2g1'], 
        y=subdata['obs_p2g1'], 
        method='pearson')
    r_ls.append(corr['r'].values[0])
print(len([r for r in r_ls if r > 0])/len(r_ls))

0.7428571428571429


## L|L

In [80]:
# 在 L|L 上的设定的转移系数和被试反应的转移系数的相关状况
DataLL = behdata[['gen_p1g1', 'obs_p1g1','subject']].dropna()
DataLL_sublevel = DataLL.groupby(['subject']).transform(lambda x: (x - x.mean()) / x.std())
DataLL_sublevel["subject"] = DataLL["subject"]
DataLL_sublevel = DataLL_sublevel.rename(
    columns={'gen_p1g1':'Generative p(L|L)',
             'obs_p1g1':'Observed p(L|L)'})
print(pg.corr(
    x=DataLL_sublevel['Generative p(L|L)'], 
    y=DataLL_sublevel['Observed p(L|L)'], 
    method='pearson'))

            n         r         CI95%     p-val     BF10     power
pearson  1121  0.118704  [0.06, 0.18]  0.000068  103.567  0.978798


In [81]:
r_ls = []
for subid in behdata["subject"].unique():
    subdata = DataLL[DataLL["subject"] == subid]
    corr = pg.corr(
        x=subdata['gen_p1g1'], 
        y=subdata['obs_p1g1'], 
        method='pearson')
    r_ls.append(corr['r'].values[0])
print(len([r for r in r_ls if r > 0])/len(r_ls))

0.7428571428571429
